# Principal Component Analysis

In this exercise sheet we look into how to compute and apply a Principal Component Analysis (PCA).

In [1]:
import numpy as np
import mllab.pca
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

### Task 3.1 - PCA Implementation

Implement the `pca()` function below using either a singular value decomposition or an eigenvector decomposition.

In [ ]:
def pca(x, q):
    """
    Compute principal components and the coordinates.
    
    Parameters
    ----------
    
    x: (n, d) NumPy array
    q: int
       The number of principal components to compute.
       Has to be less than `p`.

    Returns
    -------
    
    Vq: (d, q) NumPy array, orthonormal vectors (column-wise)
    xq: (n, q) NumPy array, coordinates for x (row-wise)
    """
    # You code goes here
    w, v = np.linalg.eigh(np.dot(x.T,x))
    Vq = v[:,x.shape[1]-q:]
    xq = np.dot(Vq.T,(x-np.mean(x,axis=0)).T).T
    return Vq, xq

### Task 3.2 - Toy 4D Example

We start by loading our toy example. The data is stored as a Numpy array, it is a $2585\times 5$ matrix. The last component of each row is the label, the first four components are the coordinates in 4D. Each label is an integer from  $\{0, 1, 2, 3, 4\}$.

The data contains a noisy 2D plane which is embded into 4D. We would like to represent the data in its _intrinsic_ 2D form.

In [ ]:
import numpy as np
import mllab.pca

pca_toy_4d = np.load("data/pca_toy_4d.npy")
x = pca_toy_4d[:, :-1]  # 4D coordinates
y = pca_toy_4d[:, -1]  # labels

**Let us plot slices from this 4D data.**

We provide a helper function for this:

In [ ]:
# Show documentation
mllab.pca.plot_toy_slice?

In [ ]:
# Your code goes here
mllab.pca.plot_toy_slice(x,y,2)

We want to remove the noise and recover the 2D information.

**PCA transformation for $q=2$, and plot.**

Now we can compute the 2D dimensional representation of `x` using PCA.

In [ ]:
# Compute transformation
Vq, xq = pca(x,2)

And then plot the coordinates, which are two dimensional. We provide a helper function for this task. Let us check how to use it:

In [ ]:
mllab.pca.plot_toy_2d?

In [ ]:
# Plot
mllab.pca.plot_toy_2d(xq, y)

Hopefully you appreciate the result.

**Check non-linear transformation.**

Let us see how PCA handles a non-linear transformation. To test this we map our data into 3D by keeping the y-axis as the new z-axis and bending x-coordinate onto an ellipse.

In [ ]:
mllab.pca.map_on_ellipse?

In [ ]:
# Map on ellipse in 3D using mallab.pca.map_on_ellipse
num_samples = 5
axis1 = np.random.rand(num_samples)*6+.1
degrees = np.random.randint(360, size=num_samples)

print(degrees)

xq_transformed = np.zeros((num_samples, xq.shape[0], 3))
for i in range(num_samples):
    xq_transformed[i] = mllab.pca.map_on_ellipse(xq,axis1[i],1,degrees[i])

In [ ]:
mllab.pca.plot_toy_3d?

In [ ]:
%matplotlib notebook
# Plot 3D using mllab.pca.plot_toy_3d
for i in range(num_samples):
    mllab.pca.plot_toy_3d(xq_transformed[i],y)

**(Remeber to stop the interactive plot by pressing the shutdown icon!)**

Now apply PCA to our transformed data and plot the result as before.

In [ ]:
%matplotlib inline
# Transform back to 2D
for i in range(num_samples):
    Vq, xq = pca(xq_transformed[i], 2)
    print("Angle={}, a={}".format(degrees[i], axis1[i]))
    mllab.pca.plot_toy_2d(xq,y)
# Plot with mllab.pca.plot_toy_2d

Could be worse, but undeniably discomforting. Try different axes lengths and gap sizes of the ellipse. What do you observe?

### Task 3.3 - PCA on Iris

First compute the singular values of the Iris dataset, then check how many percent of the variance the first two principal components capture.

In [ ]:
from sklearn.datasets import load_iris
iris = load_iris()
iris_x = iris['data']
iris_y = iris['target']

In [ ]:
# Compute singular values
import numpy.matlib
s = np.linalg.svd(iris_x, full_matrices=False, compute_uv = False)

# Plot captured variance for q=1,2
print(np.power(np.linalg.norm(np.triu(np.matlib.repmat(np.flip(s,0),4,1), -1),axis=1),2)/np.dot(s,s))


**Plot 2D PCA.**

Now apply PCAFalse and compute the first two principal components. Plot the projected 2D data in a scatter plot such that the three labels are recognizable. What do you observe?

In [ ]:
# Scatter plot 2d
Vq, xq = pca(iris_x, 2)
mllab.pca.plot_toy_2d(xq,iris_y)

In [ ]:
def plot_1d_iris(a, b, c):
    """Show a 1D plot of three 1D datasets a, b and c.
    
    Top to bottom plotted in order is a, b, c."""
    left = min(x.min() for x in (a, b, c))
    right = max(x.max() for x in (a, b, c))
    for i, (x, c) in enumerate(((a, 'red'), (b, 'blue'), (c, 'green'))):
        plt.hlines(i * .3, left, right, linestyles='dotted', colors=[(.8,.8,.8,1)])
        plt.eventplot(x, colors=c, linewidths=.5, linelengths=.25, lineoffsets=(2 - i) * .3)
    plt.axis('off')

# Plot 1d
Vq, xq = pca(iris_x, 1)
xq = xq.reshape(-1)
plot_1d_iris(xq[:50],xq[50:100],xq[100:])

**Build linear SVM classifier for Iris.**

In [ ]:
# Your classifier here
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix

class irisClassifier:
    def __init__(self, dim):
        self.clf1_ = LinearSVC(random_state=0)
        self.clf2_ = LinearSVC(random_state=1)
        self.dim_ = dim
        
    def fit(self, iris_x, iris_y):
        Vq, xq = pca(iris_x, self.dim_)
        self.clf1_.fit(xq, iris_y > 1.5)
        mask = self.clf1_.predict(xq) == 0
        print(mask)
        self.clf2_.fit(xq[mask], iris_y[mask] > 0.5)
    
    def predict(self, x):
        Vq, xq = pca(x, self.dim_)
        prediction = self.clf1_.predict(xq)
        mask = prediction == 0
        prediction = prediction * 2
        prediction[mask] = self.clf2_.predict(xq[mask])
        return prediction
        
# Print accuracy on 1, 2, and 4 dimensions.

clf = irisClassifier(2)
clf.fit(iris_x,iris_y)

c = confusion_matrix(clf.predict(iris_x), iris_y)
print(c.trace()/iris_y.size)


## Pedestrian Classification

### Task 3.4

In [2]:
input_file_path = "data/pca_ped_25x50.mat"

__Rread the file above into a NumPy array__

In [3]:
# Your code here
import scipy.io
ped_data = scipy.io.loadmat(input_file_path)

__Get the training data out__

In [4]:
# Your code here
x = np.append(ped_data['ped_train_int_25x50'].T[1:].T,ped_data['garb_train_int_25x50'].T[1:].T,axis=0)
y = np.append(ped_data['ped_train_int_25x50'].T[:1].T,ped_data['garb_train_int_25x50'].T[:1].T)

x_test = np.append(ped_data['ped_test_int_25x50'].T[1:].T,ped_data['garb_test_int_25x50'].T[1:].T,axis=0)
y_test = np.append(ped_data['ped_test_int_25x50'].T[:1].T,ped_data['garb_test_int_25x50'].T[:1].T)

__Normalize the data to the range [0, 1]__

In [5]:
# Your code here
x_n = (x-np.amin(x))/(np.amax(x)-np.amin(x))
x_test_n = (x_test-np.amin(x_test))/(np.amax(x_test)-np.amin(x_test))

__Write a function to plot an image__

In [ ]:
# Your code here
def plot_im(idx,x):
    plt.imshow(x[idx].reshape(25,50).T, vmin=0, vmax=1, cmap='gray')
    plt.show()

In [ ]:
# Plot samples
for i in np.random.randint(0,1500,10):
    plot_im(i,x_n)
for i in np.random.randint(1500,3000,10):
    plot_im(i,x_n)

### Task 3.5 - Eigenpedestrians

In [ ]:

# Compute PCA
pca = PCA(25*50)
pca.fit(x_n)

In [ ]:
# Plot some eigenpedestrians

components_n = (pca.components_ - np.amin(pca.components_))/(np.amax(pca.components_)-np.amin(pca.components_))

for i in list(range(20))+list(range(50,60))+list(range(100,110)):
    plot_im(i,components_n)

### Task 3.6 - Linear SVM classifier

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
import multiprocessing as mp


# Compute scores of linear SVM when using increasing q
pca = PCA(25*50)
x_transformed = pca.fit_transform(x_n)
x_test_transformed = pca.transform(x_test_n)

def fitSVM(i):
    print("Fitting model using {} principal components".format(i))
    clf = LinearSVC(random_state=0, dual=False)
    clf.fit(x_transformed[:,:i],y) 
    score_test = accuracy_score(y_test, clf.predict(x_test_transformed[:,:i]))
    score_train = accuracy_score(y, clf.predict(x_transformed[:,:i]))
    return (score_test, score_train, i)

pool = mp.Pool(processes=4)
results = pool.map(fitSVM, range(10,1250,25))

results.sort(key=lambda triple: triple[2])  

      




In [ ]:
# Plot q vs scores
plt.scatter(list(range(10,1250,25)),[triple[0] for triple in results], c='red')
plt.scatter(list(range(10,1250,25)),[triple[1] for triple in results], c='blue')

### Task 3.7 - HOG features

In [6]:
# Implement HOG features
import hog

**To test you implementation test data is availabe.**

The array `image` is the input, and `steps` contains the values of the inner variables of the HOG algorithm.

In [7]:
image, steps = mllab.pca.hog_test_data()

import timeit
start_time = timeit.default_timer()

hogs = hog.extract(x_n.reshape(3000,25,50))
hogs_test = hog.extract(x_test_n.reshape(1000,25,50))
   

elapsed = timeit.default_timer() - start_time
print(elapsed)

Number of bins: 9
Cell size: 8
Block size: 2
Unsigned directions: True
Clip value: 0.2
0.5664230880065588


** Repeat task 3.6 with the HOG features.**

In [10]:
# Your code here
hogs = np.zeros(shape=(x_n.shape[0], 360))
hogs_test = np.zeros(shape=(x_test_n.shape[0], 360))
for i in range(x_n.shape[0]):
    hogs[i] = hog.extract(x_n[i].reshape(25,50))
for i in range(x_test_n.shape[0]) :
    hogs_test[i] = hog.extract(x_test_n[i].reshape(25,50))


pca = PCA(hogs[0].size)    
hogs_transformed = pca.fit_transform(hogs)
hogs_test_transformed = pca.transform(hogs_test)

def fitSVM(i):
    print("Fitting model using {} principal components".format(i))
    clf = LinearSVC(random_state=0, dual=False)
    clf.fit(hogs_transformed[:,:i],y) 
    score_test = accuracy_score(y_test, clf.predict(hogs_test_transformed[:,:i]))
    score_train = accuracy_score(y, clf.predict(hogs_transformed[:,:i]))
    return (score_test, score_train, i)

pool = mp.Pool(processes=4)
results = pool.map(fitSVM, range(10,360,10))

results.sort(key=lambda triple: triple[2])

plt.scatter(list(range(10,360,10)),[triple[0] for triple in results], c='red')
plt.scatter(list(range(10,360,10)),[triple[1] for triple in results], c='blue')

NameError: name 'mp' is not defined